The following jupyter notebook contains the computation for the stratified expected value included in the appendix example in the paper Alonso et al. 
Note that the model used for the characteristic function is modified to fit the requirements in the stratified expected value where the considered agents in each stratum has different battery capacities than agent $i$. This is the only difference with the model used for the Nested Shapley values.

The stratified expected method is presented in 

Cremers, S., Robu, V., Zhang, P., Andoni, M., Norbu, S., & Flynn, D. (2023). Efficient methods for approximating the Shapley value for asset sharing in energy communities. Applied Energy, 331, 120328. https://doi.org/https://doi.org/10.1016/j.apenergy.2022.120328 


In [1]:
import numpy as np
import os
import pandas as pd
import pytz
from src.ClusteringAlgorithm import GenerateArray
from src.appendix_model import model_p2p_stratified

In [2]:
# Directory with data
current_directory = os.path.normpath(os.path.join(os.getcwd(), ".."))
file_path_data = os.path.join(current_directory,r"data_appendix")

# Input data
start_date_str = "2019-6-11"
end_date_str = "2019-6-12"

N=12 # number of agents
np.random.seed(42)  # You can choose any number as a seed
houses_pv = np.random.choice(range(1, N + 1), int(N/2+2), replace=False)
houses_pv = [f"{i}" for i in houses_pv]
n_houses_pv = len(houses_pv)
PV_average = (n_houses_pv * 5) / N # calculate the average solar capacity

houses_bat = ['12', '10', '8', '9']
n_houses_bat = len(houses_bat)
S_average = (n_houses_bat * 4) / N # calculate the average storage capacity

Import and preprocess the demand data

In [4]:
def import_demand(file_demand,N):
    date_format_str = '%Y-%m-%d %H:%M:%S%z'  # '2019-12-06 14:00:00+00:00' format
    demand = pd.read_csv(file_demand, index_col=0,
                         parse_dates=[0], date_format=date_format_str)

    utc_tz = pytz.UTC  # just used to ensure matching the dates with the index
    start_date = pd.to_datetime(start_date_str, format='%Y-%m-%d').tz_localize(utc_tz)
    end_date = pd.to_datetime(end_date_str, format='%Y-%m-%d').tz_localize(utc_tz)
    demand.index = demand.index.to_pydatetime()
    demand = demand[demand.columns[:N]]
    demand = demand[(demand.index >= start_date) & (demand.index < end_date)]
    
    return demand
    
file_demand = os.path.join(file_path_data,"demand_Jan_365days.csv")
demand = import_demand(file_demand,N)

Functions

In [6]:
def generate_data_dict_stratified(file_path_data, start_date_str, end_date_str, demand, current_agents, S_average=None, PV_average=None, house_bat_bool=None, house_pv_bool=None):
    # Case decentralised
    list_houses = current_agents.copy()
    if house_pv_bool is True:
        if "i" in list_houses:
            list_houses_pv = list_houses
            capacity_pv = list(np.full(len(list_houses_pv[:-1]), PV_average))
            capacity_pv.append(5) # todo: convert to variable
        else:
            list_houses_pv = list_houses
            capacity_pv = list(np.full(len(list_houses_pv), PV_average))
    else:
        if "i" in list_houses:
            list_houses_pv = list_houses[:-1]
            capacity_pv = list(np.full(len(list_houses_pv), PV_average))
        else:
            list_houses_pv = list_houses
            capacity_pv = list(np.full(len(list_houses_pv), PV_average))

    if house_bat_bool is True:
        if "i" in list_houses:
            list_houses_bat = list_houses
        else:
            list_houses_bat = list_houses
    else:
        if "i" in list_houses:
            list_houses_bat = list_houses[:-1]
        else:
            list_houses_bat = list_houses

    # transforming dates to align with data
    utc_tz = pytz.UTC  # just used to ensure matching the dates with the index
    start_date = pd.to_datetime(start_date_str, format='%Y-%m-%d').tz_localize(utc_tz)
    end_date = pd.to_datetime(end_date_str, format='%Y-%m-%d').tz_localize(utc_tz)

    # Get spot prices
    file_path = os.path.join(file_path_data, r"dayahead_Jan_365days.csv")
    date_format_str = '%Y-%m-%d %H:%M:%S%z'  # '2019-12-06 14:00:00+00:00' format
    P_spot_df = pd.read_csv(file_path, index_col=0,
                            parse_dates=[0], date_format=date_format_str)  # to make sure the date is read properly
    P_spot_df.index = P_spot_df.index.to_pydatetime() # convert to a datetime format required for the model
    P_spot_df = P_spot_df[["day ahead price (p/kWh)"]]  # get only price in pences/kWh
    P_spot_df_ = P_spot_df[(P_spot_df.index >= start_date) & (P_spot_df.index < end_date)]
    # Convert the dataframe P_spot_df_ to dictionary for data input for the function model_p2p()
    P_spot = P_spot_df_.to_dict()

    # Get demand
    demand_ = demand.stack()  # Set time and household as index
    # Convert the dataframe to dictionary
    P_demand = demand_.to_dict()

    # Get solar profiles, we assume the PV profile is the same for each house given that they are located close to each other
    file_path = os.path.join(file_path_data, r"solar_profile_scenarios_yearly.csv")
    PV_df = pd.read_csv(file_path, index_col=0,
                        parse_dates=[0], date_format=date_format_str)
    PV_df.index = PV_df.index.to_pydatetime() # convert to a datetime format required for the model
    scn = "1"
    PV_df = PV_df[[scn]]  # Select just one scenario, the data is prepared for several scenarios
    PV_df_ = PV_df[(PV_df.index >= start_date) & (PV_df.index < end_date)]
    # Convert the dataframe to dictionary
    PV = PV_df_.to_dict()

    # Set T
    list_T = P_spot_df_.index.to_list()

    # Parameter PV_cap
    PV_cap = {f"{key}":capacity_pv[i] for i, key in enumerate(list_houses_pv)}

    # Scalars (single value parameters)
    Psi = 1 - 0.076  # Losses (assume a loss of 7.6% through the local network, Luth)
    Mu_c = 0.96  # Charging efficiency
    Mu_d = 0.96  # Discharging efficiency

    # Case with shared resources
    #Smax = 4  # capacity batteries [kWh] # It can also be changes to be similar to parameter PV_cap where you specify the capacity of each battery
    # Case with no shared resources
    Smax = S_average
    Alpha = S_average * 1.5/4  # charging rate 2.5 kW -> 1.25 kWh/hour at constant rate
    Beta =  S_average * 1.5/4  # discharging rate 2.5 kW -> 1.25 kWh/hour at constant rate
    Smin = Smax * 0.2  # minimum state of charge of batteries at all times
    S_init = Smax * 0.5  # initial state of charge of the battery

    if "i" in list_houses_bat:
        Smax_dict = {agent: Smax for agent in list_houses_bat[:-1]}
        Alpha_dict = {agent: Alpha for agent in list_houses_bat[:-1]}
        Beta_dict = Alpha_dict.copy()
        Smin_dict = {agent: Smin for agent in list_houses_bat[:-1]}
        Sinit_dict = {agent: S_init for agent in list_houses_bat[:-1]}

        Smax_dict["i"] = 4 # add the battery capacity of the agent i mannually if it corresponds to the houses with batteries
        Alpha_dict["i"] = 1.5
        Beta_dict["i"] = 1.5
        Smin_dict["i"] = 4 * 0.2
        Sinit_dict["i"] = 4 * 0.5
    else:
        Smax_dict = {agent: Smax for agent in list_houses_bat}
        Alpha_dict = {agent: Alpha for agent in list_houses_bat}
        Beta_dict = Alpha_dict.copy()
        Smin_dict = {agent: Smin for agent in list_houses_bat}
        Sinit_dict = {agent: S_init for agent in list_houses_bat}

    c_FFR = 1000

    # Construct data dictionary
    data = {  # always start with None and then dictionary
        None: {  # names of the keys equal to the name of the parameteres in the model
            'H': {None: list_houses},  # providing data for set H
            'H_pv': {None: list_houses_pv},  # providing data for set H_pv
            "H_bat": {None: list_houses_bat},  # providing data for set H_bat
            "T": {None: list_T},  # providing datetime for set T
            # Parameters
            'P_spot': P_spot['day ahead price (p/kWh)'],
            "PV": PV[scn],
            "PV_cap": PV_cap,
            "Dem": P_demand,
            # Scalars
            "Psi": {None: Psi},
            "Mu_c": {None: Mu_c},
            "Mu_d": {None: Mu_d},
            "Alpha": Alpha_dict,
            "Beta": Beta_dict,
            "Smax": Smax_dict,
            "Smin": Smin_dict,
            "S_init": Sinit_dict,
            "c_FFR": {None: c_FFR}
        }}

    return data

### Case 1: Different agent contribution

In [7]:
# Initialise matrix C to save the costs for different agents in different stratums
C = np.zeros((N, 2, N)) # strata, costs, agents

agents_list = [f"{i}" for i in demand.columns.tolist()] # list of agents

for i, agent in enumerate(agents_list): # For each agent
    house_bat_bool = False
    S_average = (n_houses_bat * 4) / (N)
    house_pv_bool = False
    PV_average = (n_houses_pv * 5) / (N)  # Included also in generation of data!

    # Calculate the average demand profile without agent i
    demand_average = demand.drop(agent, axis=1)
    demand_average = demand_average.mean(axis=1)

    if agent in houses_bat:
        house_bat_bool = True
        S_average = ((n_houses_bat - 1) * 4) / (N) # reduce the capacity of the rest of agents if the house has already capacity

    if agent in houses_pv:
        house_pv_bool = True
        PV_average = ((n_houses_pv - 1) * 5) / (N)  # reduce the capacity of the rest of agents if the house has already capacity

    for j in range(N): # Go through every stratum
        if j !=0: # Skip calculation of v(emptyset)
            print(f"\nAgent {agent} Stratum {j} Cost c(j)")
            # Calculate cost for S({1,...,j}) with average demand
            demand_stratum = pd.concat([demand_average] * j, axis=1)
            demand_stratum.columns = [f"{l}" for l in range(1, j+1)]
            current_agents = demand_stratum.columns.to_list()
            data_c_j = generate_data_dict_stratified(file_path_data, start_date_str, end_date_str, demand_stratum, current_agents, S_average, PV_average, house_bat_bool, house_pv_bool)
            instance_c_j = model_p2p_stratified(data_c_j)
            C[j, 0, i] = instance_c_j.objective_function.expr()

            # Calculate cost for S({1,...,j}) with average demand U {i}
            print(f"\nAgent {agent} Stratum {j} Cost c(jUi)")
            demand_stratum["i"] = demand[agent]
            current_agents.append("i")
            data_c_j_i = generate_data_dict_stratified(file_path_data, start_date_str, end_date_str, demand_stratum, current_agents, S_average, PV_average, house_bat_bool, house_pv_bool)
            instance_c_j_i = model_p2p_stratified(data_c_j_i)
            C[j, 1, i] = instance_c_j_i.objective_function.expr()
        else:
            print(f"\nAgent {agent} Stratum {j} Cost c(jUi) (Empty Set j)")
            # Calculate cost for S({1,...,j}) with average demand U {i}
            demand_stratum = demand[agent].to_frame()
            demand_stratum.columns = ["i"]
            current_agents = ["i"]

            data_c_j_i = generate_data_dict_stratified(file_path_data, start_date_str, end_date_str, demand_stratum, current_agents, S_average, PV_average, house_bat_bool, house_pv_bool)
            instance_c_j_i = model_p2p_stratified(data_c_j_i)
            C[j, 1, i] = instance_c_j_i.objective_function.expr()

# Activate to save the Cost Matrix, C
#C_2d = C.reshape(-1, C.shape[2]) # every 2 rows it identifies a stratum the first row is the c(j) and the second the c(jUi)
#np.savetxt(r"C_stratified.csv", C_2d, delimiter=",", fmt="%f")

# Calculate SEV_i
SEV_i = np.zeros(N)
for i, agent in enumerate(agents_list):
    c_j_i = C[:, 1, i]
    c_j = C[:, 0, i]

    sum_cost = 0
    for j in range(len(c_j)):
        sum_cost += c_j_i[j] - c_j[j]

    SEV_i[i] = 1/N * sum_cost


Agent 1 Stratum 0 Cost c(jUi) (Empty Set j)
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-08
Read LP format model from file C:\Users\raquelal\AppData\Local\Temp\tmpke6wn91h.pyomo.lp
Reading time = 0.00 seconds
x242: 241 rows, 241 columns, 529 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 241 rows, 241 columns and 529 nonzeros
Model fingerprint: 0xf16e481c
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-02, 1e+00]
Presolve removed 241 rows and 241 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8348785e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.834878543e+

Now, we need to normalise it to the real characteristic function of the entire coalition. This value was calculated from the cases in `appendix_nestedshapley.ipynb`

In [10]:
c_N = 272.940594

In [11]:
total_SEV_i = np.sum(SEV_i)
SEV = np.zeros(len(SEV_i))
for i in range(len(SEV_i)):
    SEV[i] = SEV_i[i]*c_N/total_SEV_i

In [12]:
print("The stratified expected values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {SEV[i].round(1)}")

The stratified expected values of the agents are: 

Agent 1: 12.4
Agent 2: 8.4
Agent 3: 32.4
Agent 4: 45.1
Agent 5: 20.8
Agent 6: 13.6
Agent 7: 10.7
Agent 8: 13.7
Agent 9: 5.8
Agent 10: 31.2
Agent 11: 41.1
Agent 12: 37.8


### Case 2: Equal agent contribution

In [ ]:
# Initialise matrix C to save the costs for different agents in different stratums
C = np.zeros((N, 2, N)) # strata, costs, agents

agents_list = [f"{i}" for i in demand.columns.tolist()] # list of agents

for i, agent in enumerate(agents_list): # For each agent
    S_average = 4 # they all have 4 kWh
    PV_average = 5 # They all have 5 kW

    # Calculate the average demand profile without agent i
    demand_average = demand.drop(agent, axis=1)
    demand_average = demand_average.mean(axis=1)

    for j in range(N): # Go through every stratum
        if j !=0: # Skip calculation of v(emptyset)
            print(f"\nAgent {agent} Stratum {j} Cost c(j)")
            # Calculate cost for S({1,...,j}) with average demand
            demand_stratum = pd.concat([demand_average] * j, axis=1)
            demand_stratum.columns = [f"{l}" for l in range(1, j+1)]
            current_agents = demand_stratum.columns.to_list()
            data_c_j = generate_data_dict_stratified(file_path_data, start_date_str, end_date_str, demand_stratum, current_agents, S_average, PV_average, house_bat_bool=None, house_pv_bool=None)
            instance_c_j = model_p2p_stratified(data_c_j)
            C[j, 0, i] = instance_c_j.objective_function.expr()

            # Calculate cost for S({1,...,j}) with average demand U {i}
            print(f"\nAgent {agent} Stratum {j} Cost c(jUi)")
            demand_stratum["i"] = demand[agent]
            current_agents.append("i")
            data_c_j_i = generate_data_dict_stratified(file_path_data, start_date_str, end_date_str, demand_stratum, current_agents, S_average, PV_average, house_bat_bool=None, house_pv_bool=None)
            instance_c_j_i = model_p2p_stratified(data_c_j_i)
            C[j, 1, i] = instance_c_j_i.objective_function.expr()
        else:
            print(f"\nAgent {agent} Stratum {j} Cost c(jUi) (Empty Set j)")
            # Calculate cost for S({1,...,j}) with average demand U {i}
            demand_stratum = demand[agent].to_frame()
            demand_stratum.columns = ["i"]
            current_agents = ["i"]

            data_c_j_i = generate_data_dict_stratified(file_path_data, start_date_str, end_date_str, demand_stratum, current_agents, S_average, PV_average, house_bat_bool=None, house_pv_bool=None)
            instance_c_j_i = model_p2p_stratified(data_c_j_i)
            C[j, 1, i] = instance_c_j_i.objective_function.expr()

# Activate to save the Cost Matrix, C
#C_2d = C.reshape(-1, C.shape[2]) # every 2 rows it identifies a stratum the first row is the c(j) and the second the c(jUi)
#np.savetxt(r"C_stratified.csv", C_2d, delimiter=",", fmt="%f")

# Calculate SEV_i
SEV_i = np.zeros(N)
for i, agent in enumerate(agents_list):
    c_j_i = C[:, 1, i]
    c_j = C[:, 0, i]

    sum_cost = 0
    for j in range(len(c_j)):
        sum_cost += c_j_i[j] - c_j[j]

    SEV_i[i] = 1/N * sum_cost